# 第5章 再び，Deep Learning！

## ニューラルネットワークの基礎理論を理解した上で，もう一度デモを動かしてみよう

11. フレームワークを使う(TensorFlow + Keras)
ここまで見てきたように，頑張れば自力でニューラルネットワークを実装することができる．
しかし，とても大変だったことがわかると思う．
さらに，今，Deep Learningの研究を行っている人が多数いる．
その人達と成果物を共有しようとするとなると，各自が自作したプログラムでは議論が難しい．

そこで，Deep Learningを動かす枠組み（フレームワーク）が多く開発されている．
ここでは，それらの一つであるGoogleが開発したTensorFlowを紹介する．
TensorFlowはニューラルネットワークを動かすというよりは，テンソル（3次元以上の多次元行列）の演算を行うために開発されたものである．
もちろん，Deep Learningのために開発されたのであるが，ニューラルネットワークは行列の演算を多く行う．
そのため，行列演算のために，しかも並列計算が可能になるように開発されたものがTensorFlowである．

さらに，　KerasはTensorFlowのフロントエンドあり，Deep Learningのモデルを短い記述で実装できる．
一般的に，TensorFlowでDeep Learningを実装しようとすると，Kerasを用いる．

TensorFlow + Kerasを使えば簡単に実装できる．
ニューラルネットワークの構造を変えたり，活性化関数を変更したり，最適化のアルゴリズムを変更したり，とにかくいろいろできる．
さらに，グラフを表示したり，ネットワークの出力を可視化したり，とにかく機能が豊富である．

そのため，せっかく簡単に記述できるにも関わらず，とても複雑になり，初心者にはわかりにくいサンプルプログラムをが多い．
そこで，下記のプログラムはこれまで説明してきたXORを，TensorFlow + Kerasで短く，わかりやすく記述することを目的とする．

## 一つずつ動かしてみよう

In [ ]:
# TensorFlow をセットアップする
import tensorflow as tf

# MNIST（0から9の数字画像）セットを読み込む
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 機械学習モデルを構築する
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

# モデルの構成
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

# 学習
model.fit(x_train, y_train, epochs=5)

# モデルの評価を行う
model.evaluate(x_test,  y_test, verbose=2)

# テストデータに対する予測を行う
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

probability_model(x_test[:5])

これまでニューラルネットワークを自作することで動作させてきた

In [ ]:
# TensorFlow をセットアップする
import tensorflow as tf

## 文字認識の結果を確認してみよう

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

predictions = probability_model(x_test[:5])

# 予測と正解を可視化して比べてみる
for index, prediction in enumerate(predictions):
    print(f'予測：{np.argmax(prediction)} 正解：{y_test[index]}')
    ax = plt.subplot(3, 3, index + 1)
    plt.imshow(x_test[index] * 255, cmap="gray")
    plt.show()

### 参考文献
- [初心者のための TensorFlow 2.0 入門](https://www.tensorflow.org/tutorials/quickstart/beginner?hl=ja)
- [TensorFlowのチュートリアルをコメント付けながら実行してみた（初心者のための_TensorFlow_2_0_入門）](https://qiita.com/penpenta/items/ee45f58d416c656639aa)

[講義資料に戻る](https://github.com/crotsu/Deep_Learning_Starting_with_Examples)

このページは[このリンク](https://colab.research.google.com/github/crotsu/Deep_Learning_Starting_with_Examples/blob/main/chap5_deeplearning/chap5_document.ipynb)よりgoogle colaboratoryで動作させることができる．  